<a href="https://colab.research.google.com/github/Mano-hard/Options-mispricing-/blob/main/Options_mispricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install requests pandas --quiet


In [6]:
import requests
import pandas as pd

symbol = "BANKNIFTY"

url = f"https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nseindia.com/option-chain"
}

# Create a session to handle cookies
session = requests.Session()
session.get("https://www.nseindia.com", headers=headers)  # Necessary to get cookies
response = session.get(url, headers=headers)
data = response.json()

# Check spot price
spot_price = data['records']['underlyingValue']
print("✅ Spot Price:", spot_price)

# Get all option data
option_data = data['records']['data']
expiry = data['records']['expiryDates'][0]  # nearest expiry

# Filter for nearest expiry
filtered = [item for item in option_data if item['expiryDate'] == expiry]

# Preview one
print("✅ First Option Entry:", filtered[0])


✅ Spot Price: 55617.6
✅ First Option Entry: {'strikePrice': 48000, 'expiryDate': '28-Aug-2025', 'CE': {'strikePrice': 48000, 'expiryDate': '28-Aug-2025', 'underlying': 'BANKNIFTY', 'identifier': 'OPTIDXBANKNIFTY28-08-2025CE48000.00', 'openInterest': 2655, 'changeinOpenInterest': 5, 'pchangeinOpenInterest': 0.18867924528301888, 'totalTradedVolume': 28, 'impliedVolatility': 0, 'lastPrice': 7748.55, 'change': -376.1999999999998, 'pChange': -4.630296316809746, 'totalBuyQuantity': 3885, 'totalSellQuantity': 3255, 'bidQty': 35, 'bidprice': 7727, 'askQty': 35, 'askPrice': 7789.85, 'underlyingValue': 55617.6}, 'PE': {'strikePrice': 48000, 'expiryDate': '28-Aug-2025', 'underlying': 'BANKNIFTY', 'identifier': 'OPTIDXBANKNIFTY28-08-2025PE48000.00', 'openInterest': 5075, 'changeinOpenInterest': 1118, 'pchangeinOpenInterest': 28.25372757139247, 'totalTradedVolume': 13461, 'impliedVolatility': 24.29, 'lastPrice': 10.85, 'change': -0.7000000000000011, 'pChange': -6.060606060606069, 'totalBuyQuantity'

In [7]:
!pip install requests pandas numpy scipy matplotlib --quiet


In [8]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")


In [9]:
def black_scholes_price(option_type, S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


In [12]:
r = 0.06  # risk-free rate
today = datetime.today()
expiry_date = datetime.strptime(expiry, "%d-%b-%Y")
T = (expiry_date - today).days / 365

rows = []

for item in option_data:
    K = item['strikePrice']

    # Call
    if 'CE' in item and item['CE'].get('lastPrice') and item['CE'].get('impliedVolatility'):
        market = item['CE']['lastPrice']
        iv = item['CE']['impliedVolatility'] / 100
        theo = black_scholes_price('call', spot_price, K, T, r, iv)
        diff = abs(market - theo)
        rows.append({'Type': 'Call', 'Strike': K, 'Market': market, 'IV': iv, 'BS_Price': round(theo,2), 'Diff': round(diff,2)})

    # Put
    if 'PE' in item and item['PE'].get('lastPrice') and item['PE'].get('impliedVolatility'):
        market = item['PE']['lastPrice']
        iv = item['PE']['impliedVolatility'] / 100
        theo = black_scholes_price('put', spot_price, K, T, r, iv)
        diff = abs(market - theo)
        rows.append({'Type': 'Put', 'Strike': K, 'Market': market, 'IV': iv, 'BS_Price': round(theo,2), 'Diff': round(diff,2)})

df = pd.DataFrame(rows)
df = df.sort_values(by='Diff', ascending=False)
df.head(10)


,Type,Strike,Market,IV,BS_Price,Diff
161,Call,57000,1730.00,0.0846,111.63,1618.37
229,Call,60000,1515.00,0.0970,0.80,1514.20
204,Call,58500,1090.00,0.0922,12.37,1077.63
119,Call,56000,1425.00,0.0790,373.80,1051.20
81,Call,55000,1692.00,0.0745,962.93,729.07
96,Call,55500,1369.40,0.0824,654.56,714.84
101,Call,55600,1307.05,0.0832,600.01,707.04
70,Call,54500,2050.90,0.0558,1346.77,704.13
105,Call,55700,1236.40,0.0827,541.57,694.83
109,Call,55800,1189.20,0.0849,501.76,687.44


from matplotlib import pyplot as plt
_df_0['Strike'].plot(kind='hist', bins=20, title='Strike')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Market'].plot(kind='hist', bins=20, title='Market')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['IV'].plot(kind='hist', bins=20, title='IV')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['BS_Price'].plot(kind='hist', bins=20, title='BS_Price')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='BS_Price', y='Diff', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['Strike'].plot(kind='line', figsize=(8, 4), title='Strike')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['Market'].plot(kind='line', figsize=(8, 4), title='Market')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['IV'].plot(kind='line', figsize=(8, 4), title='IV')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['BS_Price'].plot(kind='line', figsize=(8, 4), title='BS_Price')
plt.gca().spines[['top', 'right']].set_visible(False)